In [1]:
from flax import linen as nn
import jax
import jax.numpy as jnp
from models import VisionTransformer

from clu import metrics
from flax.training import train_state  # Useful dataclass to keep train state
from flax import struct                # Flax dataclasses
import optax                           # Common loss functions and optimizers

import matplotlib.pyplot as plt

/home/jannik/miniconda3/lib/python3.10/site-packages/jax/_src/api_util.py:190: SyntaxWarning: Jitted function has static_argnums=(0, 1, 9), but only accepts 8 positional arguments. This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has {argnums_name}={argnums}, "


In [2]:
# horizon = 10 
# dt = 0.1 
# rec_dt = 0.025  # 40 Hz data collection
# every_n = int(dt/rec_dt)

In [3]:
# from torch.utils.data import Dataset, DataLoader, random_split
# import torch

# class FieldDataset(Dataset):
#     def __init__(self, file_dir='data/robot_field_data.pt'):
#         grids, commands = torch.load(file_dir)
        
#         self.grids = grids[::every_n].to(torch.float32)
#         self.commands = commands[::every_n][:,[0,1,5]].to(torch.float32)  # get u_x, u_y, u_w

#     def __len__(self):
#         return len(self.grids) - horizon

#     def __getitem__(self, idx):
#         grid = self.grids[idx]
#         commands = self.commands[idx:idx+horizon]
#         return grid, commands

In [4]:
# dataset = FieldDataset('data/impossible_obstacle.pt')

# train_ratio = 0.8

# train_size = int((train_ratio) * len(dataset))
# val_size = len(dataset) - train_size

# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# print(f'Length of training dataset: {len(train_dataset)}')
# print(f'Length of validation dataset: {len(val_dataset)}')

# batch_size = 1
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Length of training dataset: 252
Length of validation dataset: 64


In [5]:
# config = {
#     'patches': (4,4),
#     'transformer': {
#       'num_layers': 3,
#       'mlp_dim': 64,
#       'num_heads': 1,
#       'dropout_rate': 0.1,
#       'attention_dropout_rate': 0.1,
#     },
#     'solver': {
#       'dt': dt,
#       'horizon': horizon,
#     },
#     'hidden_size': 42,
#     'num_output': 6*6+6,
#     'representation_size': 64,
# }


# model = VisionTransformer(**config)

# print(model.tabulate({'params': jax.random.PRNGKey(0), 'dropout': jax.random.PRNGKey(1)}, 
#                      jnp.ones((1, 28, 28, 1)), 
#                      train=True))

In [6]:
# @struct.dataclass
# class Metrics(metrics.Collection):
#   # accuracy: metrics.Accuracy
#   loss: metrics.Average.from_output('loss')
  
# class TrainState(train_state.TrainState):
#   metrics: Metrics

# def create_train_state(module, rng, learning_rate, momentum, train=True):
#   """Creates an initial `TrainState`."""
  
#   # initialize parameters by passing a template image
#   params = module.init(rng, jnp.ones([1, 100, 100, 1]), train=train)['params'] 
  
#   tx = optax.sgd(learning_rate, momentum)
#   return TrainState.create(
#       apply_fn=module.apply, params=params, tx=tx,
#       metrics=Metrics.empty())

In [7]:
# @jax.jit
# def train_step(state, batch, train=True):
#   """Train for a single step."""
#   def loss_fn(params):
#     dropout_rng = jax.random.PRNGKey(1)
    
#     pred = state.apply_fn({'params': params},
#                             rngs={'dropout': dropout_rng},
#                             inputs=batch['inputs'], 
#                             train=train)
        
#     loss = optax.l2_loss(pred, batch['targets']).mean()
#     return loss
  
#   grad_fn = jax.grad(loss_fn)
#   grads = grad_fn(state.params)
  
#   state = state.apply_gradients(grads=grads)
#   return state

# @jax.jit
# def compute_metrics(*, state, batch, train=True):
#   dropout_rng = jax.random.PRNGKey(1)
  
#   pred = state.apply_fn({'params': state.params},
#                           rngs={'dropout': dropout_rng},
#                           inputs=batch['inputs'], 
#                           train=train)
  
#   loss = optax.l2_loss(pred, batch['targets']).mean()
  
#   metric_updates = state.metrics.single_from_model_output(loss=loss)
#   metrics = state.metrics.merge(metric_updates)
  
#   state = state.replace(metrics=metrics)
#   return state

In [8]:
# grid, _ = dataset.__getitem__(100)
# input = jnp.expand_dims(grid.numpy(), (0,-1))

# init_rng = {'params': jax.random.PRNGKey(0), 'dropout': jax.random.PRNGKey(1)}

# params = model.init(init_rng, jnp.ones([1, 100, 100, 1]), train=True)['params'] 

# output = model.apply({'params': params}, input, rngs=init_rng, train=True)
# print(output)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


[[[ 0.3654899  -0.0301541  -0.01419155]
  [ 0.36607835 -0.02743974 -0.01207794]
  [ 0.36695573 -0.02452096 -0.01001463]
  [ 0.36812302 -0.02139261 -0.00800748]
  [ 0.36958176 -0.01804929 -0.00606247]
  [ 0.371334   -0.01448542 -0.0041857 ]
  [ 0.37338242 -0.01069519 -0.0023834 ]
  [ 0.37573037 -0.00667263 -0.00066193]
  [ 0.37838173 -0.00241157  0.00097221]
  [ 0.38134107  0.00209433  0.0025124 ]]]


In [9]:
init_rng = {'params': jax.random.PRNGKey(0), 'dropout': jax.random.PRNGKey(1)}

train = True
learning_rate = 0.001
momentum = 0.9

state = create_train_state(model, init_rng, learning_rate, momentum)
del init_rng

In [10]:
from tqdm import tqdm

metrics_history = {'train_loss': []}
num_epochs = 10
epoch_iterator = range(num_epochs)

for epoch in epoch_iterator:
  data_iterator = tqdm(train_loader)
  for grids, commands in data_iterator:
    grids = jax.numpy.expand_dims(grids.numpy(), axis=-1)
    
    batch = {'inputs': grids, 'targets': commands.numpy()}
    
    state = train_step(state, batch) # get updated train state (which contains the updated parameters)
    state = compute_metrics(state=state, batch=batch) # aggregate batch metrics
    
    data_iterator.set_postfix(loss=state.metrics.loss.compute_value().value)
  
  for metric,value in state.metrics.compute().items():  # compute metrics
    metrics_history[f'train_{metric}'].append(value)    # record metrics
  state = state.replace(metrics=state.metrics.empty()) 
  
  print(f"train epoch: {epoch}, "
        f"loss: {metrics_history['train_loss'][-1]}, ")   

100%|██████████| 252/252 [00:40<00:00,  6.29it/s, loss=0.013564912] 


train epoch: 0, loss: 0.013564911670982838, 


 44%|████▍     | 111/252 [00:12<00:16,  8.76it/s, loss=0.012433872] 


KeyboardInterrupt: 